# EDA

In [9]:
import pandas as pd

df = pd.read_csv('shopee-product-matching/train.csv')

In [10]:
df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [11]:
unique_labels = df['label_group'].unique()
len(unique_labels)

11014

In [12]:
df.groupby('label_group').count().sort_values(by='posting_id', ascending=False)

,posting_id,image,image_phash,title
label_group,,,,
1163569239,51,51,51,51
159351600,51,51,51,51
994676122,51,51,51,51
3113678103,51,51,51,51
3627744656,51,51,51,51
...,...,...,...,...
2357508171,2,2,2,2
2357372960,2,2,2,2
2357221297,2,2,2,2


In [13]:
# check title length range by word
df['title'].str.split().apply(lambda x: len(x)).describe()

count    34250.000000
mean         9.414861
std          4.446719
min          1.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         61.000000
Name: title, dtype: float64

In [19]:
df['title'].nunique()

33117

In [15]:
# check the id count
df['posting_id'].nunique()

34250

In [16]:
len(df['posting_id'])

34250

In [18]:
df["image_phash"].nunique()

28735